<a href="https://colab.research.google.com/github/BrianSward/colab-work/blob/main/fake_news_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# imports and such for the learning part

import numpy as np 
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.svm import LinearSVC

In [ ]:
# installs for the web scraping

!pip install charset-normalizer
!pip install spacy
!pip install trafilatura

In [68]:
# imports for the web scraping

from bs4 import BeautifulSoup
import json
import numpy as np
import requests
from requests.models import MissingSchema
import spacy
import trafilatura

In [69]:
data = pd.read_csv("https://raw.githubusercontent.com/lutzhamel/fake-news/master/data/fake_or_real_news.csv")

Begin Data Training

In [71]:
data['fake'] = data['label'].apply(lambda x: 0 if x == "REAL" else 1)
data = data.drop("label", axis=1)

In [72]:
X, y = data["text"], data["fake"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [73]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
X_train_vectorized = vectorizer.fit_transform(X_train) 
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
clf = LinearSVC()
clf.fit(X_train_vectorized, y_train)

Lets test the accuracy of our model


In [112]:
_score = round(clf.score(X_test_vectorized, y_test), 2)
print(f'{_score*100}%')

94.0%


The cell below will accept a url, and then process it

In [109]:
user_url = input("Feed me a url!!! ")

text_data = {}
resp = requests.get(user_url)
if resp.status_code == 200:
  text_data[user_url] = resp.text

def beautifulsoup_extract_text_fallback(response_content):
        
    # Create the beautifulsoup object:
    soup = BeautifulSoup(response_content, 'html.parser')
    
    # Finding the text:
    text = soup.find_all(text=True)
    
    # Remove unwanted tag elements:
    cleaned_text = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        'style',]

    # Then we will loop over every item in the extract text and make sure that the beautifulsoup4 tag
    # is NOT in the blacklist
    for item in text:
        if item.parent.name not in blacklist:
            cleaned_text += '{} '.format(item)
            
    # Remove any tab separation and strip the text:
    cleaned_text = cleaned_text.replace('\t', '')
    return cleaned_text.strip()
    

def extract_text_from_single_web_page(url):
    
    downloaded_url = trafilatura.fetch_url(url)
    try:
        a = trafilatura.extract(downloaded_url, output_format='json', with_metadata=True, include_comments = False,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    except AttributeError:
        a = trafilatura.extract(downloaded_url, json_output=True, with_metadata=True,
                            date_extraction_params={'extensive_search': True, 'original_date': True})
    if a:
        json_output = json.loads(a)
        return json_output['text']
    else:
        try:
            resp = requests.get(url)
            # We will only extract the text from successful requests:
            if resp.status_code == 200:
                return beautifulsoup_extract_text_fallback(resp.content)
            else:
                # This line will handle for any failures in both the Trafilature and BeautifulSoup4 functions:
                return np.nan
        # Handling for any URLs that don't have the correct protocol
        except MissingSchema:
            return np.nan
          
single_url = user_url
_text = extract_text_from_single_web_page(url=single_url)
vectorized_text = vectorizer.transform([_text])

if clf.predict(vectorized_text) == [0]:
  print("with 92% confidence thats real news")

else:
  print("with 92% confidence thats fake news")

Feed me a url!!! https://www.rt.com/russia/575170-ukraine-zelensky-decolonization-law/
with 92% confidence thats fake news
